# NLTK Chatbot Training

### Scope of this chatbot
We are going to build a chatbot using deep learning techniques using a **retrieval-based** approach. The chatbot will be trained on the dataset 
which contains conversation categories (intents), patterns, and responses. The model uses a Deep Neural Network with a single hidden layer to 
classify which category the input message belongs to and then the chatbot will select a random response from the list of responses, which have 
similar meaning.

Topics the chatbot will be helpful with helping students finding answers to questions in the following topics:
- Before calculus
- Limits and continuity
- Derivatives
- Integrals

Furthermore, this is just a prototype whose functionality can be greatly expanded in topics (other than math) it can reply to, depth of conversation, answer a
plethra of questions and so on.

In [1]:
import random
from tensorflow.keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout
from keras.models import Sequential
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [2]:
DIR_NAME = '/Users/robed/Desktop/calc-tutor-bot/chatbot/'
nltk.data.path.append('./nltk_data/')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /Users/robed/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/robed/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## Intents (Input Data)

In [3]:
train = pd.read_json(DIR_NAME+'data/intents.json', orient='records')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   intents  21 non-null     object
dtypes: object(1)
memory usage: 296.0+ bytes


For each intent there is information on:
- tag: Topic of conversation
- patterns: The user input
- responses: The chatbot's reply
- context: A field that correlates to the tag field

## Load json file

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open(DIR_NAME+'data/intents.json').read()
intents = json.loads(data_file)

 
for intent in intents['intents']:
    for pattern in intent['patterns']:

        tockenized_word_patterns = nltk.word_tokenize(pattern)
        words.extend(tockenized_word_patterns)
        documents.append((tockenized_word_patterns, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

## Process words and classes

In [5]:
lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [6]:
pickle.dump(words, open(DIR_NAME + '/words.pkl', 'wb'))
pickle.dump(classes, open(DIR_NAME + '/classes.pkl', 'wb'))

## Preprocessing

In [7]:
training = []

output_empty = [0] * len(classes)

for document in documents:
    bag = []

    tokenized_words = document[0]
    tokenized_words = [lemmatizer.lemmatize(word.lower()) for word in tokenized_words]

    for word in words:
        bag.append(1) if word in tokenized_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x_bags = list(training[:, 0])
train_y_output_rows = list(training[:, 1])
print("Training data created")

Training data created


## Create a Deep Neural Network

In [8]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x_bags[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y_output_rows[0]), activation='softmax'))

2022-02-27 19:39:01.623818: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, metrics=['accuracy'])

/Users/robed/Desktop/calc-tutor-bot/env/lib/python3.7/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


## Fit and save model

In [10]:
hist = model.fit(np.array(train_x_bags), np.array(train_y_output_rows), epochs=300, batch_size=10, verbose=1)
model.save('./chatbot_model/chatbot_model.h5', hist)

print("model created")

Epoch 1/300
10/10 [==============================] - 1s 4ms/step - loss: 3.0599 - accuracy: 0.0532
Epoch 2/300
10/10 [==============================] - 0s 12ms/step - loss: 3.0214 - accuracy: 0.1064
Epoch 3/300
10/10 [==============================] - 0s 4ms/step - loss: 2.9370 - accuracy: 0.1702
Epoch 4/300
10/10 [==============================] - 0s 3ms/step - loss: 2.9594 - accuracy: 0.0957
Epoch 5/300
10/10 [==============================] - 0s 4ms/step - loss: 2.8429 - accuracy: 0.1809
Epoch 6/300
10/10 [==============================] - 0s 4ms/step - loss: 2.7667 - accuracy: 0.2447
Epoch 7/300
10/10 [==============================] - 0s 4ms/step - loss: 2.7368 - accuracy: 0.2128
Epoch 8/300
10/10 [==============================] - 0s 8ms/step - loss: 2.7422 - accuracy: 0.1915
Epoch 9/300
10/10 [==============================] - 0s 6ms/step - loss: 2.6228 - accuracy: 0.2447
Epoch 10/300
10/10 [==============================] - 0s 3ms/step - loss: 2.6101 - accuracy: 0.2766
Epoch 11